Análisis de modelos con TensorFlow Model Analysis
=================================================

¿Que es TensorFlow Model Analysis?
-----------------------------------------------------

TensorFlow Model Analysis (TFMA) es una biblioteca de Python para realizar la evaluación del modelo en diferentes segmentos de datos. TFMA es una herramienta muy potente en ambientes productivos ya que realiza sus cálculos de manera distribuida sobre grandes cantidades de datos utilizando Apache Beam, lo cual lo hace altamente escalable. TFMA se puede utilizar para investigar y visualizar el rendimiento de un modelo con respecto a las características del conjunto de datos.

Para demostrar esta técnica utilizaremos el conjunto de datos del censo UCI.

Instalar TFMA
-------------

In [ ]:
!pip install tensorflow-model-analysis

### Instalación en Jupyter corriendo en su equipo

Para poder utilizar esta librería, deberá instalar las siguientes extensiones. Estas extensiones no son necesarias en Google Colab

In [1]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix 
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix 
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix 

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
2021-10-11 12:57:23.313182: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-11 12:57:23.313235: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Enabling notebook extension tensorflow_model_analysis/extension...
      - Validating: OK


Explorando un modelo
------------------

### Sobre el conjunto de datos del censo UCI

El conjunto de datos del censo de la UCI es un conjunto de datos en el que cada registro representa a una persona. Cada registro contiene 14 columnas que describen a una una sola persona, de la base de datos del censo de Estados Unidos de 1994. Esto incluye información como la edad, el estado civil y el nivel educativo. La tarea es determinar si una persona tiene un ingreso alto (definido como ganar más de $50 mil al año). Esta tarea, dado el tipo de datos que utiliza, se usa a menudo en el estudio de equidad, en parte debido a los atributos comprensibles del conjunto de datos, incluidos algunos que contienen tipos sensibles como la edad y el género, y en parte también porque comprende una tarea claramente del mundo real.

El mismo tiene las siguientes características:

- Education-Num
- Occupation
- Marital-Status
- Never-married
- Capital-Loss
- Race
- Country
- Workclass
- Age
- Education
- Hours-per-week
- Sex
- Relationship
- Capital-Gain
- Target

### Sobre el modelo

Utilizaremos un modelo entrenado en este conjunto de datos. El mismo lo puede acceder desde `datasets/uci_census/model/1`

### Utilizando TFMA

Primero necesitamos convertir nuestro modelo a un formato de evaluación:

In [1]:
import tensorflow as tf
import tensorflow_model_analysis as tfma

In [129]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Target',
                                signature_name='classification',
                                prediction_key='scores')],
    slicing_specs=[
        tfma.SlicingSpec(), # Overall
        tfma.SlicingSpec(feature_keys=["Race"]),
        tfma.SlicingSpec(feature_keys=["Occupation"])
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            binarize=tfma.BinarizationOptions(class_ids={ 'values': [0]}),
            metrics=[
                tfma.MetricConfig(class_name='BinaryAccuracy', 
                                  threshold=tfma.MetricThreshold(
                                      value_threshold=tfma.GenericValueThreshold(
                                          lower_bound={'value': 0.6}),
                                          change_threshold=tfma.GenericChangeThreshold(
                                              direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                              absolute={'value': -1e-10}))),
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='FalsePositives'),
                tfma.MetricConfig(class_name='TruePositives'),
                tfma.MetricConfig(class_name='FalseNegatives'),
                tfma.MetricConfig(class_name='TrueNegatives'),
        ])
    ])

> **Nota:** `BinarizationOptions` fué indicado en esta configuración ya que nuestro modelo predice las probabilidades para cada una de las clases disponibles en lugar de la clase más probable. Para más información sobre esta configuración puede revisar https://github.com/tensorflow/model-analysis/blob/master/g3doc/faq.md

In [130]:
eval_shared_model = tfma.default_eval_shared_model(eval_saved_model_path='./datasets/uci_census/model/1',
                                                   tags=[tf.saved_model.SERVING])

Luego, debemos indicarle a TFMA cual es nuestra variable objetivo junto con algunas especificaciones de como queremos evaluar el modelo, utilizando cuales características y computando cuales métricas.

Una vez que tenemos configuramos el modelo, lo evaluamos

In [131]:
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config,
    data_location='datasets/uci_census/adult.tfrecord',
    output_path='datasets/uci_census/eval.tfrecord')

ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).


Visualizamos los resultados en Jupyter:

In [132]:
tfma.view.render_slicing_metrics(eval_result, slicing_column="Race")

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Race:White', 'metric…